Firstly we will import all the dependencies 

In [41]:
import json
import os
# import shutil
# import cv2
import numpy as np
# import os
# from matplotlib import pyplot as plt
# import time
# import mediapipe as mp
# import re

Finding Out the Maximum Sequence Length

***Creating Folders according to different types (Right Handedness, Left Handedness, Both, Mouth Utterance etc)***

In [42]:
bds_folder = "../BdSLW60"
numerical_data_path='../NumericalData'

In [43]:

specialTargets = {'W7','W20','W41','W44','W46','W50','W94','W98','W211','W216','W217','W356'}
PER_FRAME_FEATURE =1629
testUser = {'U4','U8'}




In [44]:
datas = []
word_lists = os.listdir(bds_folder)
for word in word_lists:
    if word.endswith('.pdf') or word.endswith('.xlsx') or word.endswith('.txt'):
        pass 
    else:
        with open(f'{bds_folder}/{word}/output1.json', 'r') as file:
            # Step 3: Parse the JSON data into a Python data structure
            datas.append(json.load(file))

In [45]:
missing_count =0
trial_count=0
specialTargets_trial_count=0
specialTargets_missing_count=0
total_frame_count=0
spcialTargets_frame_count=0

for data in datas:
    for word in data:
        for user in data[word]:
            for orientation in data[word][user]:
                 for fileName in data[word][user][orientation]:
                    
                    frameRate = data[word][user][orientation][fileName]['FrameRate']
                    no_trials = len(data[word][user][orientation][fileName]['trials'])
                    
                    #override the trial count as sometime that is missing in annotation
                    data[word][user][orientation][fileName]['no_of_trials']= no_trials           
                    
                    
                    

                    data_path = f'{numerical_data_path}/'
                    if user in testUser:
                        data_path = data_path+'Test/'
                    else:
                        data_path = data_path+'Training/'
            
                    data_path = data_path + orientation
                  
                    word_path = data_path+'/'+word
                    
                    user_path = word_path+'/'+user
                
                
                     
                    fileName_path = user_path+'/'+fileName
                    
                    
                
                    
                    
                    
                        
                    
                    
                    trial_path = fileName_path+'/'
                
                
                    i=0
                    for i in range(0,no_trials):
                        
                        
                        trial_folder = trial_path+f't{str(i)}'                                                         
                            
                        
                        starting = data[word][user][orientation][fileName]['trials'][str(i)]['starting']
                        ending = data[word][user][orientation][fileName]['trials'][str(i)]['ending']
                        
                        
                        
                        
                       #do some statistics about missing values
                        missing_in_trial=0
                        fcount=0
                        for frame in range(starting,ending+1):
                            #load the npy
                            trial_npy_path = trial_folder+'/'+str(frame)+'.npy'
                            # print(trial_npy_path)
                            npy=np.load(trial_npy_path)
                            # npy_matrix=npy.reshape(-1,PER_FRAME_FEATURE)
                            fno=1503
                            for i in range (0,42):
                                if npy[fno] <=0.0 and npy[fno+1] <=0.0 and npy[fno+2] <=0.0:
                                    
                                    missing_in_trial+=1
                                fno+=3
                            fcount+=1
                        #for loop
                        total_frame_count+=fcount
                        missing_count+=missing_in_trial
                        trial_count+=1
                        if word in specialTargets:
                            specialTargets_trial_count+=1
                            specialTargets_missing_count+=missing_in_trial
                            spcialTargets_frame_count+=fcount

                        
                       

                    # end of trial loop

print('missing points count:',missing_count,' in trials: ',trial_count, ' Avg per trial: ',missing_count/trial_count,' Avg per frame: ',missing_count/total_frame_count,' Avg per HAND POINT: ',missing_count/(total_frame_count*42))
print('missing points IN TARGET WORD TRIALS: ',specialTargets_missing_count,' in trials: ',specialTargets_trial_count,' Avg per trial: ',specialTargets_missing_count/specialTargets_trial_count,' Avg per Frame: ',specialTargets_missing_count/spcialTargets_frame_count, 'AVG per HAND POINT: ',specialTargets_missing_count/(spcialTargets_frame_count*42))
print('Missing hand point percentage (other than special word): ',(missing_count-specialTargets_missing_count)/((total_frame_count-spcialTargets_frame_count)*42))                    


missing points count: 5881449  in trials:  9307  Avg per trial:  631.938218545181  Avg per frame:  17.399243851858863  Avg per HAND POINT:  0.4142677107585444
missing points IN TARGET WORD TRIALS:  1192947  in trials:  1823  Avg per trial:  654.3867251782775  Avg per Frame:  19.89140113050873 AVG per HAND POINT:  0.4736047888216364
Missing hand point percentage (other than special word):  0.4014694881606583
